In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

import seaborn as sns
from imblearn.over_sampling import SMOTE
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

## Using TensorFlow backend.

In [ ]:
data = pd.read_csv("data_preprocessed/data_after_preprocess.csv", encoding ='utf-8')
# record = pd.read_csv("data_set/credit_record.csv", encoding ='utf-8')

In [ ]:
plt.rcParams['figure.facecolor'] = 'white'

## Feature Engineering
-  Response Variable

In [ ]:
begin_month=pd.DataFrame(record.groupby(["ID"])["MONTHS_BALANCE"].agg(min))
begin_month=begin_month.rename(columns={'MONTHS_BALANCE':'begin_month'})
new_data=pd.merge(data,begin_month,how="left",on="ID") #merge to record data

- Generally, users in risk should be in 3%, thus I choose users who overdue for more than 60 days as target risk users. Those samples are marked as '1', else are '0'.

In [ ]:
record['dep_value'] = None
record['dep_value'][record['STATUS'] =='3']='Yes'
record['dep_value'][record['STATUS'] =='4']='Yes'
record['dep_value'][record['STATUS'] =='5']='Yes' 

In [ ]:
cpunt=record.groupby('ID').count()
cpunt['dep_value'][cpunt['dep_value'] > 0]='Yes'
cpunt['dep_value'][cpunt['dep_value'] == 0]='No'
cpunt = cpunt[['dep_value']]
new_data=pd.merge(new_data,cpunt,how='inner',on='ID')
new_data['target']=new_data['dep_value']
new_data.loc[new_data['target']=='Yes','target']=1
new_data.loc[new_data['target']=='No','target']=0

In [ ]:
new_data

In [ ]:
# Calculate Applicant Age and Years of Working
new_data["APPLICATION_AGE"] = -new_data["DAYS_BIRTH"] // 365
new_data["YEARS_OF_WORKING"] = -new_data["DAYS_EMPLOYED"] // 365

In [ ]:
# Drop rows where either Years_of_Working is negative
new_data = new_data[
    (new_data['YEARS_OF_WORKING'] >= 0)
    ]

In [ ]:
new_data

In [ ]:
new_data = new_data.drop(['DAYS_BIRTH', 'DAYS_EMPLOYED'], axis=1)

In [ ]:
new_data = new_data.drop(['dep_value'], axis=1)

In [ ]:
new_data = new_data.drop(['begin_month'], axis=1)

In [ ]:
new_data

## normalization 
- AMT_INCOME_TOTAL 

In [ ]:
# For Z-score normalization
scaler_zscore = StandardScaler()
zscore_cols = ['AMT_INCOME_TOTAL']
# Create new column names for normalized features
zscore_cols_scaled = [col + '_zscore' for col in zscore_cols]
# Transform and assign to new columns
new_data[zscore_cols_scaled] = scaler_zscore.fit_transform(new_data[zscore_cols])

In [ ]:
new_data = new_data.drop(['AMT_INCOME_TOTAL'], axis=1)

In [ ]:
new_data

In [ ]:
# Apply filtering conditions
filtered_application_df = new_data[
    (new_data["APPLICATION_AGE"] >= 21) &
    new_data["CODE_GENDER"].notnull() &
    new_data["FLAG_OWN_CAR"].notnull() &
    new_data["FLAG_OWN_REALTY"].notnull() &
    new_data["CNT_CHILDREN"].notnull() &
    new_data["NAME_INCOME_TYPE"].notnull() &
    new_data["NAME_EDUCATION_TYPE"].notnull() &
    new_data["NAME_FAMILY_STATUS"].notnull() &
    new_data["NAME_HOUSING_TYPE"].notnull()
    ]

In [ ]:
filtered_application_df['FLAG_MOBIL'].value_counts()

In [ ]:
filtered_application_df['target'] = new_data['target'].astype(int)

In [ ]:
filtered_application_df['target'].value_counts()

In [ ]:
filtered_application_df

In [ ]:
numeric_cols = ['CNT_CHILDREN', 'FLAG_MOBIL', 'APPLICATION_AGE', 'YEARS_OF_WORKING',
                'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL',
                'CNT_FAM_MEMBERS','AMT_INCOME_TOTAL_zscore','target']

# Calculate correlation matrix
correlation_matrix = filtered_application_df[numeric_cols].corr()

# Create heatmap
plt.figure(figsize=(9, 8))
sns.heatmap(correlation_matrix,
            annot=True,  # Show numbers
            cmap='coolwarm',  # Red/Blue color scheme
            center=0,  # Center the colorbar at 0
            fmt='.3f',  # 2 decimal places
            square=True)  # Make cells square

plt.title('Correlation Heatmap of Numeric Features')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
new_data = new_data.drop(['FLAG_MOBIL'], axis=1)

In [ ]:
numeric_cols = ['CNT_CHILDREN', 'APPLICATION_AGE', 'YEARS_OF_WORKING',
                'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL',
                'CNT_FAM_MEMBERS','AMT_INCOME_TOTAL_zscore','target']

# Calculate correlation matrix
correlation_matrix = filtered_application_df[numeric_cols].corr()

# Create heatmap
plt.figure(figsize=(9, 8))
sns.heatmap(correlation_matrix,
            annot=True,  # Show numbers
            cmap='coolwarm',  # Red/Blue color scheme
            center=0,  # Center the colorbar at 0
            fmt='.3f',  # 2 decimal places
            square=True)  # Make cells square

plt.title('Correlation Heatmap of Numeric Features')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## Features

In [ ]:
# drop null and NAN
filtered_application_df.dropna()
filtered_application_df = filtered_application_df.mask(filtered_application_df == 'NULL').dropna()

In [ ]:
filtered_application_df

In [ ]:
ivtable=pd.DataFrame(filtered_application_df.columns,columns=['variable'])
ivtable['IV']=None
namelist = ['target','ID']

for i in namelist:
    ivtable.drop(ivtable[ivtable['variable'] == i].index, inplace=True)

- Define calc_iv function to calculate -> https://www.kaggle.com/code/puremath86/iv-woe-starter-for-python Information Value and WOE Value

# Binary Features

In [ ]:
# Calculate information value
def calc_iv(df, feature, target, pr=False):
    lst = []
    df[feature] = df[feature].fillna("NULL")

    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append([feature,                                                        # Variable
                    val,                                                            # Value
                    df[df[feature] == val].count()[feature],                        # All
                    df[(df[feature] == val) & (df[target] == 0)].count()[feature],  # Good (think: Fraud == 0)
                    df[(df[feature] == val) & (df[target] == 1)].count()[feature]]) # Bad (think: Fraud == 1)

    data = pd.DataFrame(lst, columns=['Variable', 'Value', 'All', 'Good', 'Bad'])
    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bad'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
    data['Distribution Bad'] = data['Bad'] / data['Bad'].sum()
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])

    data = data.replace({'WoE': {np.inf: 0, -np.inf: 0}})

    data['IV'] = data['WoE'] * (data['Distribution Good'] - data['Distribution Bad'])

    data = data.sort_values(by=['Variable', 'Value'], ascending=[True, True])
    data.index = range(len(data.index))

    if pr:
        print(data)
        print('IV = ', data['IV'].sum())

    iv = data['IV'].sum()
    print('This variable\'s IV is:',iv)
    print(df[feature].value_counts())
    return iv, data

In [ ]:
def convert_dummy(df, feature,rank=0):
    pos = pd.get_dummies(df[feature], prefix=feature)
    mode = df[feature].value_counts().index[rank]
    biggest = feature + '_' + str(mode)
    pos.drop([biggest],axis=1,inplace=True)
    df.drop([feature],axis=1,inplace=True)
    df=df.join(pos)
    return df

In [ ]:
def get_category(df, col, binsnum, labels, qcut = False):
    if qcut:
        localdf = pd.qcut(df[col], q = binsnum, labels = labels) # quantile cut
    else:
        localdf = pd.cut(df[col], bins = binsnum, labels = labels) # equal-length cut

    localdf = pd.DataFrame(localdf)
    name = 'gp' + '_' + col
    localdf[name] = localdf[col]
    df = df.join(localdf[name])
    df[name] = df[name].astype(object)
    return df

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

- Gender

In [ ]:
filtered_application_df

In [ ]:
filtered_application_df['CODE_GENDER'] = filtered_application_df['CODE_GENDER'].replace(['F','M'],[0,1])
print(filtered_application_df['CODE_GENDER'].value_counts())
iv, data = calc_iv(filtered_application_df,'CODE_GENDER','target')
ivtable.loc[ivtable['variable']=='CODE_GENDER','IV']=iv
data.head()

In [ ]:
filtered_application_df

In [ ]:
numeric_cols = ['CNT_CHILDREN', 'CODE_GENDER' ,'APPLICATION_AGE', 'YEARS_OF_WORKING',
                'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL',
                'CNT_FAM_MEMBERS','AMT_INCOME_TOTAL_zscore','target']

# Calculate correlation matrix
correlation_matrix = filtered_application_df[numeric_cols].corr()

# Create heatmap
plt.figure(figsize=(9, 8))
sns.heatmap(correlation_matrix,
            annot=True,  # Show numbers
            cmap='coolwarm',  # Red/Blue color scheme
            center=0,  # Center the colorbar at 0
            fmt='.3f',  # 2 decimal places
            square=True)  # Make cells square

plt.title('Correlation Heatmap of Numeric Features')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

- Having a car or not

In [ ]:
filtered_application_df['FLAG_OWN_CAR'] = filtered_application_df['FLAG_OWN_CAR'].replace(['N','Y'],[0,1])
print(filtered_application_df['FLAG_OWN_CAR'].value_counts())
iv, data=calc_iv(filtered_application_df,'FLAG_OWN_CAR','target')
ivtable.loc[ivtable['variable']=='FLAG_OWN_CAR','IV']=iv
data.head()

In [ ]:
filtered_application_df

In [ ]:
numeric_cols = ['CNT_CHILDREN', 'FLAG_OWN_CAR','CODE_GENDER' ,'APPLICATION_AGE', 'YEARS_OF_WORKING',
                'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL',
                'CNT_FAM_MEMBERS','AMT_INCOME_TOTAL_zscore','target']

# Calculate correlation matrix
correlation_matrix = filtered_application_df[numeric_cols].corr()

# Create heatmap
plt.figure(figsize=(9, 8))
sns.heatmap(correlation_matrix,
            annot=True,  # Show numbers
            cmap='coolwarm',  # Red/Blue color scheme
            center=0,  # Center the colorbar at 0
            fmt='.3f',  # 2 decimal places
            square=True)  # Make cells square

plt.title('Correlation Heatmap of Numeric Features')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

- Having house reality or not¶

In [ ]:
filtered_application_df['FLAG_OWN_REALTY'] = filtered_application_df['FLAG_OWN_REALTY'].replace(['N','Y'],[0,1])
print(filtered_application_df['FLAG_OWN_REALTY'].value_counts())
iv, data=calc_iv(filtered_application_df,'FLAG_OWN_REALTY','target')
ivtable.loc[ivtable['variable']=='FLAG_OWN_REALTY','IV']=iv
data.head()

In [ ]:
numeric_cols = ['CNT_CHILDREN','FLAG_OWN_REALTY', 'FLAG_OWN_CAR','CODE_GENDER' ,'APPLICATION_AGE', 'YEARS_OF_WORKING',
                'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL',
                'CNT_FAM_MEMBERS','AMT_INCOME_TOTAL_zscore','target']

# Calculate correlation matrix
correlation_matrix = filtered_application_df[numeric_cols].corr()

# Create heatmap
plt.figure(figsize=(9, 8))
sns.heatmap(correlation_matrix,
            annot=True,  # Show numbers
            cmap='coolwarm',  # Red/Blue color scheme
            center=0,  # Center the colorbar at 0
            fmt='.3f',  # 2 decimal places
            square=True)  # Make cells square

plt.title('Correlation Heatmap of Numeric Features')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
filtered_application_df = filtered_application_df.drop(['FLAG_MOBIL'], axis=1)

In [ ]:
filtered_application_df

- Having a phone or not

In [ ]:
safe_filtered_application_df = filtered_application_df

In [ ]:
filtered_application_df['FLAG_PHONE']=filtered_application_df['FLAG_PHONE'].astype(int)
print(filtered_application_df['FLAG_PHONE'].value_counts(normalize=True,sort=False))
filtered_application_df.drop(filtered_application_df[filtered_application_df['FLAG_PHONE'] == 'nan' ].index, inplace=True)
iv, data=calc_iv(new_data,'FLAG_PHONE','target')
ivtable.loc[ivtable['variable']=='FLAG_PHONE','IV']=iv
data.head()

In [ ]:
filtered_application_df

- Having an email or not

In [ ]:
print(filtered_application_df['FLAG_EMAIL'].value_counts(normalize=True,sort=False))
filtered_application_df['FLAG_EMAIL']=filtered_application_df['FLAG_EMAIL'].astype(int)
iv, data=calc_iv(new_data,'FLAG_EMAIL','target')
ivtable.loc[ivtable['variable']=='FLAG_EMAIL','IV']=iv
data.head()

- Having a Work Phone or not

In [ ]:
filtered_application_df['FLAG_WORK_PHONE']=filtered_application_df['FLAG_WORK_PHONE'].astype(str)
iv, data = calc_iv(filtered_application_df,'FLAG_WORK_PHONE','target')
new_data.drop(filtered_application_df[filtered_application_df['FLAG_WORK_PHONE'] == 'nan' ].index, inplace=True)
ivtable.loc[ivtable['variable']=='FLAG_WORK_PHONE','IV']=iv
data.head()

In [ ]:
filtered_application_df

## Continuous Variables
- Children Numbers

In [ ]:
filtered_application_df.loc[filtered_application_df['CNT_CHILDREN'] >= 2,'CNT_CHILDREN']='2More'
print(filtered_application_df['CNT_CHILDREN'].value_counts(sort=False))

In [ ]:
iv, data=calc_iv(filtered_application_df,'CNT_CHILDREN','target')
ivtable.loc[ivtable['variable']=='CNT_CHILDREN','IV']=iv
data.head()

In [ ]:
filtered_application_df = convert_dummy(filtered_application_df,'CNT_CHILDREN')

In [ ]:
filtered_application_df

## Age
- Bucketing Continuous Variables

In [ ]:
# new_data['Age']=-(new_data['DAYS_BIRTH'])//365
# print(new_data['Age'].value_counts(bins=10,normalize=True,sort=False))
# new_data['Age'].plot(kind='hist',bins=20,density=True)

In [ ]:
# new_data = get_category(new_data,'Age',5, ["lowest","low","medium","high","highest"])
# iv, data = calc_iv(new_data,'gp_Age','target')
# ivtable.loc[ivtable['variable']=='DAYS_BIRTH','IV'] = iv
# data.head()

In [ ]:
# new_data = convert_dummy(new_data,'gp_Age')

## Age
- Z-score Continuous Variables

In [ ]:
# For Z-score normalization
scaler_zscore = StandardScaler()
zscore_cols = ['APPLICATION_AGE']
# Create new column names for normalized features
zscore_cols_scaled = [col + '_zscore' for col in zscore_cols]
# Transform and assign to new columns
filtered_application_df[zscore_cols_scaled] = scaler_zscore.fit_transform(filtered_application_df[zscore_cols])

In [ ]:
filtered_application_df = filtered_application_df.drop(['APPLICATION_AGE'], axis=1)

In [ ]:
filtered_application_df

## Working Years 
- Equal-length Bucketing

In [ ]:
# new_data['worktm']=-(new_data['DAYS_EMPLOYED'])//365
# new_data[new_data['worktm']<0] = np.nan # replace by na
# new_data['DAYS_EMPLOYED']
# new_data['worktm'].fillna(new_data['worktm'].mean(),inplace=True) #replace na by mean
# print(new_data['worktm'].value_counts(bins=20,normalize=True,sort=False))
# new_data['worktm'].plot(kind='hist',bins=20,density=True)

In [ ]:
# new_data = get_category(new_data,'worktm',5, ["lowest","low","medium","high","highest"])
# iv, data=calc_iv(new_data,'gp_worktm','target')
# ivtable.loc[ivtable['variable']=='DAYS_EMPLOYED','IV']=iv
# data.head()

In [ ]:
# new_data = convert_dummy(new_data,'gp_worktm')

## Working Years 
- Min-Max

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# วิธีใช้ Min-Max
scaler = MinMaxScaler()
filtered_application_df['YEARS_OF_WORKING_normalized'] = scaler.fit_transform(filtered_application_df[['YEARS_OF_WORKING']])

In [ ]:
filtered_application_df

- Family Size

In [ ]:
filtered_application_df['CNT_FAM_MEMBERS'] = filtered_application_df['CNT_FAM_MEMBERS'].astype(int)

In [ ]:
filtered_application_df

In [ ]:
filtered_application_df['CNT_FAM_MEMBERS'].value_counts(sort=False)

In [ ]:
filtered_application_df['CNT_FAM_MEMBERS']=filtered_application_df['CNT_FAM_MEMBERS'].astype(int)
filtered_application_df['CNT_FAM_MEMBERS']=filtered_application_df['CNT_FAM_MEMBERS']
filtered_application_df['CNT_FAM_MEMBERS']=filtered_application_df['CNT_FAM_MEMBERS'].astype(object)
filtered_application_df.loc[filtered_application_df['CNT_FAM_MEMBERS']>=3,'CNT_FAM_MEMBERS']='3more'
iv, data=calc_iv(filtered_application_df,'CNT_FAM_MEMBERS','target')
ivtable.loc[ivtable['variable']=='CNT_FAM_MEMBERS','IV']=iv
data.head()

In [ ]:
filtered_application_df = convert_dummy(filtered_application_df,'CNT_FAM_MEMBERS')

In [ ]:
filtered_application_df

### Categorical Features
- Income Type

In [ ]:
print(filtered_application_df['NAME_INCOME_TYPE'].value_counts(sort=False))
print(filtered_application_df['NAME_INCOME_TYPE'].value_counts(normalize=True,sort=False))
filtered_application_df.loc[filtered_application_df['NAME_INCOME_TYPE']=='Pensioner','NAME_INCOME_TYPE']='State servant'
filtered_application_df.loc[filtered_application_df['NAME_INCOME_TYPE']=='Student','NAME_INCOME_TYPE']='State servant'
iv, data=calc_iv(filtered_application_df,'NAME_INCOME_TYPE','target')
ivtable.loc[ivtable['variable']=='NAME_INCOME_TYPE','IV']=iv
data.head()

In [ ]:
filtered_application_df = convert_dummy(filtered_application_df,'NAME_INCOME_TYPE')

In [ ]:
safe_filtered_application_df_2 = filtered_application_df

## Occupation Type

In [ ]:
filtered_application_df.loc[(filtered_application_df['OCCUPATION_TYPE']=='Cleaning staff') | (filtered_application_df['OCCUPATION_TYPE']=='Cooking staff') | (filtered_application_df['OCCUPATION_TYPE']=='Drivers') | (filtered_application_df['OCCUPATION_TYPE']=='Laborers') | (filtered_application_df['OCCUPATION_TYPE']=='Low-skill Laborers') | (filtered_application_df['OCCUPATION_TYPE']=='Security staff') | (filtered_application_df['OCCUPATION_TYPE']=='Waiters/barmen staff'),'OCCUPATION_TYPE']='Laborwk'
filtered_application_df.loc[(filtered_application_df['OCCUPATION_TYPE']=='Accountants') | (filtered_application_df['OCCUPATION_TYPE']=='Core staff') | (filtered_application_df['OCCUPATION_TYPE']=='HR staff') | (filtered_application_df['OCCUPATION_TYPE']=='Medicine staff') | (filtered_application_df['OCCUPATION_TYPE']=='Private service staff') | (filtered_application_df['OCCUPATION_TYPE']=='Realty agents') | (filtered_application_df['OCCUPATION_TYPE']=='Sales staff') | (filtered_application_df['OCCUPATION_TYPE']=='Secretaries'),'OCCUPATION_TYPE']='officewk'
filtered_application_df.loc[(filtered_application_df['OCCUPATION_TYPE']=='Managers') | (filtered_application_df['OCCUPATION_TYPE']=='High skill tech staff') | (filtered_application_df['OCCUPATION_TYPE']=='IT staff'),'OCCUPATION_TYPE']='hightecwk'
print(filtered_application_df['OCCUPATION_TYPE'].value_counts())
iv, data=calc_iv(filtered_application_df,'OCCUPATION_TYPE','target')
ivtable.loc[ivtable['variable']=='OCCUPATION_TYPE','IV']=iv
data.head() 

In [ ]:
filtered_application_df = convert_dummy(filtered_application_df,'OCCUPATION_TYPE')

In [ ]:
filtered_application_df

## House Type

In [ ]:
iv, data=calc_iv(filtered_application_df,'NAME_HOUSING_TYPE','target')
ivtable.loc[ivtable['variable']=='NAME_HOUSING_TYPE','IV']=iv
data.head()

In [ ]:
filtered_application_df = convert_dummy(filtered_application_df,'NAME_HOUSING_TYPE')

In [ ]:
filtered_application_df

## Education

In [ ]:
filtered_application_df.loc[filtered_application_df['NAME_EDUCATION_TYPE']=='Academic degree','NAME_EDUCATION_TYPE']='Higher education'
iv, data=calc_iv(filtered_application_df,'NAME_EDUCATION_TYPE','target')
ivtable.loc[ivtable['variable']=='NAME_EDUCATION_TYPE','IV']=iv
data.head()

In [ ]:
filtered_application_df = convert_dummy(filtered_application_df,'NAME_EDUCATION_TYPE')

In [ ]:
filtered_application_df

## Marriage Condition

In [ ]:
filtered_application_df['NAME_FAMILY_STATUS'].value_counts(normalize=True,sort=False)

In [ ]:
iv, data=calc_iv(filtered_application_df,'NAME_FAMILY_STATUS','target')
ivtable.loc[ivtable['variable']=='NAME_FAMILY_STATUS','IV']=iv
data.head()

In [ ]:
filtered_application_df = convert_dummy(filtered_application_df,'NAME_FAMILY_STATUS')

In [ ]:
filtered_application_df

In [ ]:
filtered_application_df.to_csv('data_after_preprocess.csv')

## IV、WOE：Concept and Application

Weight of Evidence(WoE):  

$$wo{e_i} = \ln {{{P_{yi}}} \over {{P_{ni}}}} = \ln {{{y_i}/{y_s}} \over {{n_i}/{n_s}}}$$
$wo{e_i}$ is the I category's WOE value. ${{P_{yi}}}$ is the proportion of the positive samples in this category to all positive samples.   ${{P_{ni}}}$ is the ratio of negative samples (${{n_i}}$) in this class to all negative samples (${{n_s}}$).

Information Value (IV):  
$$I{V_i} = ({P_{yi}} - {P_{ni}}) \times wo{e_i}$$  
The IV values of the various types are the difference between the conditional positive rate and the conditional negative rate multiplied by the WOE value of the variable. The total IV value of the variable can be understood as the weighted sum of the conditional positive rate and the conditional negative rate difference:
$$IV = \sum\limits_i^n {I{V_i}} $$  

The IV value measures the variable's ability to predict.


Relationship between IV value and predictive power

| IV| Ability to predict | 
|:------|:------:| 
| <0.02 | Almost no predictive power | 
|0.02~0.1 |weak predictive power|
|0.1~0.3|Moderate predictive power|
|0.3~0.5|Strong predictive power|
|>0.5|Predictive power is too strong, need to check variables| 

In [ ]:
ivtable=ivtable.sort_values(by='IV',ascending=False)
ivtable.loc[ivtable['variable']=='DAYS_BIRTH','variable']='agegp'
ivtable.loc[ivtable['variable']=='DAYS_EMPLOYED','variable']='worktmgp'
ivtable.loc[ivtable['variable']=='inc','variable']='incgp'
ivtable

# Algorithms
- Split Dataset

In [ ]:
new_data

In [ ]:
Y = new_data['target']
X = new_data[['Gender','Reality','ChldNo_1', 'ChldNo_2More','wkphone',
              'gp_Age_high', 'gp_Age_highest', 'gp_Age_low',
              'gp_Age_lowest','gp_worktm_high', 'gp_worktm_highest',
              'gp_worktm_low', 'gp_worktm_medium','occyp_hightecwk',
              'occyp_officewk','famsizegp_1', 'famsizegp_3more',
              'houtp_Co-op apartment', 'houtp_Municipal apartment',
              'houtp_Office apartment', 'houtp_Rented apartment',
              'houtp_With parents','edutp_Higher education',
              'edutp_Incomplete higher', 'edutp_Lower secondary','famtp_Civil marriage',
              'famtp_Separated','famtp_Single / not married','famtp_Widow']]

+ Using Synthetic Minority Over-Sampling Technique(`SMOTE`) to overcome sample imbalance problem.

In [ ]:
Y = Y.astype('int')
X_balance,Y_balance = SMOTE().fit_resample(X,Y)
X_balance = pd.DataFrame(X_balance, columns = X.columns)

+ After over sampling, the number between 1 and 0 is balanced. It can be seen from the confusion matrix.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_balance,Y_balance,
                                                    stratify=Y_balance, test_size=0.3,
                                                    random_state = 10086)

In [ ]:
print(X_train)

## Logistic Regression   

$$\log ({p \over {1 - p}}) = {\beta _0} + {\beta _1}{x_1} +  \cdot  \cdot  \cdot  + {\beta _q}{x_q}$$

In [ ]:
model = LogisticRegression(C=0.8,
                           random_state=0,
                           solver='lbfgs')
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

print('Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print(pd.DataFrame(confusion_matrix(y_test,y_predict)))

sns.set_style('white')
class_names = ['0','1']
plot_confusion_matrix(confusion_matrix(y_test,y_predict),
                      classes= class_names, normalize = True,
                      title='Normalized Confusion Matrix: Logistic Regression')


## Decision Tree

In [ ]:
model = DecisionTreeClassifier(max_depth=12,
                               min_samples_split=8,
                               random_state=1024)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

print('Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print(pd.DataFrame(confusion_matrix(y_test,y_predict)))

plot_confusion_matrix(confusion_matrix(y_test,y_predict),
                      classes=class_names, normalize = True,
                      title='Normalized Confusion Matrix: CART')

## Random Forest   



<center>
    <img style="border-radius: 0.3125em;
    box-shadow: 0 2px 4px 0 rgba(34,36,38,.12),0 2px 10px 0 rgba(34,36,38,.08);" 
    src="https://d1rwhvwstyk9gu.cloudfront.net/2019/03/Random-Forest-Algorithm.jpg">
    <br>
    <div style="color:orange; border-bottom: 1px solid #d9d9d9;
    display: inline-block;
    color: #999;
    padding: 2px;">Random Forest</div>
</center>

In [ ]:
model = RandomForestClassifier(n_estimators=250,
                               max_depth=12,
                               min_samples_leaf=16
                               )
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

print('Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print(pd.DataFrame(confusion_matrix(y_test,y_predict)))

plot_confusion_matrix(confusion_matrix(y_test,y_predict),
                      classes=class_names, normalize = True,
                      title='Normalized Confusion Matrix: Ramdom Forests')

## SVM


<center>
    <img style="border-radius: 0.3125em;
    box-shadow: 0 2px 4px 0 rgba(34,36,38,.12),0 2px 10px 0 rgba(34,36,38,.08);" 
    src="https://i.loli.net/2019/11/13/fryWG5al7OPHDiA.gif">
    <br>
    <div style="color:orange; border-bottom: 1px solid #d9d9d9;
    display: inline-block;
    color: #999;
    padding: 2px;">Support Vector Machine</div>
</center>

In [ ]:
model = svm.SVC(C = 0.8,
                kernel='linear')
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

print('Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print(pd.DataFrame(confusion_matrix(y_test,y_predict)))

plot_confusion_matrix(confusion_matrix(y_test,y_predict),
                      classes=class_names, normalize = True,
                      title='Normalized Confusion Matrix: SVM')

## Showing important features:

In [ ]:
# def plot_importance(classifier, x_train, point_size = 25):
#     '''plot feature importance'''
#     values = sorted(zip(x_train.columns, classifier.feature_importances_), key = lambda x: x[1] * -1)
#     imp = pd.DataFrame(values,columns = ["Name", "Score"])
#     imp.sort_values(by = 'Score',inplace = True)
#     sns.scatterplot(x = 'Score',y='Name', linewidth = 0,
#                     data = imp,s = point_size, color='red').set(
#         xlabel='importance',
#         ylabel='features')
# 
# plot_importance(model, X_train,20)   